In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

import importlib
import tifffile as tiff
from scipy.signal import chirp, find_peaks, peak_widths


import cv2
from tqdm import tqdm
from numpy import linalg as LA
from scipy.ndimage.measurements import center_of_mass

## Label statemment

#### Location
- Area:Mouse 1 Center In Center (10) #
- Area:Mouse 1 Center In Close Arm (11) #
- Area:Mouse 1 Center In Open Arm (12) #
 
- AreaStay:Mouse 1 Nose In Close Arm while TailBase In Open Arm (12)
- AreaStay:Mouse 1 Nose In Open Arm while TailBase In Close Arm (11)
 
#### Action
- Mouse 1 Grooming (31)
- Mouse 1 HeadDip [None] (32) #
- Mouse 1 SAP [Center In Close Arm] (33)
- Mouse 1 SAP [Center In Open Arm] (34)
- Mouse 1 SAP [None] (35)

In [4]:
# to define neuron type
# similarity
#----------------------------------------------------------------------------------
def similarity(x, y):
    simi = 2 * np.dot(x, y) / (LA.norm(x) ** 2 + LA.norm(y) ** 2)
    return simi



# shuffle
#----------------------------------------------------------------------------------
def random_shuffle(X):
    new_X = []
    for i in range(X.shape[1]):
        x = X[:, i].copy()
        shuffle(x)
        new_X.append(x)
    return np.vstack(new_X).T


# collect samples
#----------------------------------------------------------------------------------
def collect_samples(arr, sample_size, n_samples):
    samples = np.zeros((n_samples, sample_size), np.int32)

    for sample_n in range(0, n_samples):
        sample = get_sample(arr,
                            n_iter=sample_n,
                            sample_size=sample_size)
        samples[sample_n] = sample

    return samples


# get samples
#----------------------------------------------------------------------------------

def get_sample(arr, n_iter=None, sample_size=10, fast=True):
    return np.random.choice(arr, sample_size, replace=False)


# define ON/OFF neuron
#----------------------------------------------------------------------------------
def clsfy(B, C):
    b = B.copy()
    simi = similarity(B, C)
    x = LA.norm(b) ** 2 + LA.norm(C) ** 2
    bs = collect_samples(b, len(b), 5000)
    a = 2 * np.dot(C, bs.T) / x
    if simi >= np.percentile(a, 99.95):
        neuron_type = 'ON'
    elif simi <= np.percentile(a, 0.05):
        neuron_type = 'OFF'
    else:
        neuron_type = 'Other'
    return simi, a, neuron_type



# to check on/off neuron of certain status
#----------------------------------------------------------------------------------
def status_ONOFF(df, which_status_to_check_ON_OFF, ON_OFF, fps):
    
    # set up which_status_to_check_ON_OFF
    
    if which_status_to_check_ON_OFF == 'close':
        which_status_to_check_ON_OFF = str(11)
    elif which_status_to_check_ON_OFF == 'open':
        which_status_to_check_ON_OFF = str(12)
    else:
        raise ValueError('Not approved status. choose from: open, close')

    frame_label_list = df['Frame_Label'].values.tolist()
    
    # to separate status label, eg:20214111
    n = 2
    
    label_list = []
    for line in frame_label_list:
        line = str(line)
        line_list= [line[i:i+n] for i in range(0, len(line), n)]
        if which_status_to_check_ON_OFF in line_list:
            label_list.append(1)
        else:
            label_list.append(0)
            
    label_list_array = np.asarray(label_list)
    
    # get dataframe array; drop last column: frame_label
    df_array = df.iloc[:, :-1].to_numpy()
    # get n_neuron
    T, n_neuron = df.iloc[:, :-1].shape
    
    # get on/off neuron type
    neuron_type_list = []
    if sum(label_list_array == 1) / fps >= 4:
        for i in range(n_neuron):
            simi, a, t = clsfy(label_list_array, df_array[:, i])
            neuron_type_list.append(t)
            
    # get on/off neurom dataframe
    df_columns  = df.iloc[:, :-1].columns
    new_df_column = []
    for i in range(len(neuron_type_list)):
        if neuron_type_list[i] == ON_OFF:
            neuron_keep = df_columns[i]
            new_df_column.append(neuron_keep)

    if 'Frame_Label' in new_df_column:
        new_df_column.remove('Frame_Label')
        
    new_df = df[new_df_column]
    new_df['Frame_Label'] = frame_label_list
        
    #shape(frames, neurons+frame_label)
    return(new_df, new_df_column)

In [5]:
# to keep certain status frame interval
#----------------------------------------------------------------------------------
def check_frame_interval_with_label(df, which_status_to_check_frame):
    
    # set up which_status_to_check_frame
    
    if which_status_to_check_frame == 'close':
        which_status_to_check_frame = str(11)
    elif which_status_to_check_frame == 'open':
        which_status_to_check_frame = str(12)
    else:
        raise ValueError('Not approved status. choose from: open, close')

    frame_label_list = df['Frame_Label'].values.tolist()
    
    # to separate status label, eg:20214111
    n = 2
    
    check_frame_label_list = []
    for line in frame_label_list:
        line = str(line)
        line_list= [line[i:i+n] for i in range(0, len(line), n)]
        if which_status_to_check_frame in line_list:
            check_frame_label_list.append(1)
        else:
            check_frame_label_list.append(0)       

    df_keep = df.copy()
    df_keep['status'] = check_frame_label_list
    df_keep = df_keep[(df_keep['status'] == 1)]
    df_keep = df_keep.drop('status', axis = 1)
    #df_keep = df_keep.drop('Frame_Label', axis = 1) 
    # for whole session, frame_label doesn't need to be kept
    # but for on/off, it need to be kept
    
    print('the shape of the dataframe in the status is:', df_keep.shape)
    
    # shape(frames, neurons+frame_label)
    return(df_keep)



# calculate event rate & amplitude
#----------------------------------------------------------------------------------
def get_coordinate(corrdinate_file_path):
    rawdata = []
    with open(corrdinate_file_path, 'r') as f:
        for i, line in enumerate(f):
            if i == 9:
                start_frame = int(line.split(':')[1])
            if i == 20:
                temp = line.split('\t')
                left, right, top, bottom = float(temp[0].split(':')[2]), float(temp[1].split(':')[1]), \
                                           float(temp[2].split(':')[1]), float(temp[3].split(':')[1].split('\n')[0])
            lines = line.replace('"', '').replace("'", "").split('   ')
            try:
                rawdata.append([int(lines[0]), int(lines[1]), int(lines[2]),
                                float(lines[5]), float(lines[6]), float(lines[9])])
            except:
                pass
    rawdata = np.array(rawdata)
    return (rawdata, start_frame, left, right, top, bottom)



# distinguish prefered side of open arm or close arm, also duration in each side
#----------------------------------------------------------------------------------
def prefered_side(original_df, which_status_df_to_split, corrdinate_file_path, fps):
    
    if which_status_df_to_split.shape[0] == 0:
        prefered_df = which_status_df_to_split
        non_prefered_df = which_status_df_to_split
        x1 = np.nan
        x2 = np.nan
    else:
        coor, start_frame, left, right, top, bottom = get_coordinate(corrdinate_file_path)
        center = [(left + right) / 2, (top + bottom) / 2]

        a = np.asarray(np.arange(which_status_df_to_split.shape[0])/fps * 29.97 + (150-(coor[0, 0]-start_frame)))
        # just in case the recording is not to 600s
        new_a = a[(a <= len(coor))]
        coordinate = coor[np.floor(new_a).astype('int'), 1:3]

        # just in case the recording is not to 600s
        which_status_df_to_split_kept = which_status_df_to_split.head(len(new_a))
        
        if coordinate[0].max()-coordinate[0].min() > coordinate[1].max()-coordinate[1].min():
            # if this, then the mouse is along right&left arm, becaseu it's motion along x is wider than y
            x1 = which_status_df_to_split_kept[(coordinate[:, 0] > center[0])].shape[0]/fps
            x2 = which_status_df_to_split_kept[(coordinate[:, 0] < center[0])].shape[0]/fps
            if x1 > x2:
                prefered_df = which_status_df_to_split_kept[(coordinate[:, 0] > center[0])]
                non_prefered_df = which_status_df_to_split_kept[(coordinate[:, 0] < center[0])]
            else:
                prefered_df = which_status_df_to_split_kept[(coordinate[:, 0] < center[0])]
                non_prefered_df = which_status_df_to_split_kept[(coordinate[:, 0] > center[0])]
        else:
            # else, then the mouse is along up&down arm, 
            x1 = which_status_df_to_split_kept[(coordinate[:, 1] > center[1])].shape[0]/fps
            x2 = which_status_df_to_split_kept[(coordinate[:, 1] < center[1])].shape[0]/fps
            if x1 > x2:
                prefered_df = which_status_df_to_split_kept[(coordinate[:, 1] > center[1])]
                non_prefered_df = which_status_df_to_split_kept[(coordinate[:, 1] < center[1])]
            else:
                prefered_df = which_status_df_to_split_kept[(coordinate[:, 1] < center[1])]
                non_prefered_df = which_status_df_to_split_kept[(coordinate[:, 1] > center[1])]
    
    # get status add original dataframe 
    print(prefered_df.shape[0],'frames are prefered')
    
    prefered_df_index = prefered_df.index.values.tolist()
    non_prefered_df_index = non_prefered_df.index.values.tolist()
    
    # re-assign original_df to other names, don't make changes to original_df
    prefer_original_df = original_df.copy()
    nonprefer_original_df = original_df.copy()
    
    prefer_original_df['prefered'] = 0
    nonprefer_original_df['non_prefered'] = 0
        
    prefer_original_df.loc[prefered_df_index, 'prefered'] = 1  
    nonprefer_original_df.loc[non_prefered_df_index, 'non_prefered'] = 1 
    
    print('prefered',np.sum(prefer_original_df['prefered']))
    print('non_prefered',np.sum(nonprefer_original_df['non_prefered']))
    
    return(x1, x2, prefered_df, non_prefered_df, prefer_original_df, nonprefer_original_df)



# to check on/off neuron of certain status
### in this case, we only check social(11)/obj(12)
#----------------------------------------------------------------------------------
def status_ONOFF_Prefered(df, status_to_check, fps, ON_OFF):
    
    # choose from 'prefered','non_prefered' for status_to_check
    label_list_array = np.asarray(df[status_to_check].values.tolist())
    frame_label_list = df['Frame_Label'].values.tolist()
    
    # get dataframe array; drop last column: frame_label, prefered, non-prefered
    df_array = df.iloc[:, :-2].to_numpy()
    # get n_neuron
    T, n_neuron = df.iloc[:, :-2].shape
    
    # get on/off neuron type
    neuron_type_list = []
    if sum(label_list_array == 1) / fps >= 4:
        for i in range(n_neuron):
            simi, a, t = clsfy(label_list_array, df_array[:, i])
            neuron_type_list.append(t)
            
    # get on/off neurom dataframe
    df_columns  = df.iloc[:, :-2].columns
    new_df_column = []
    for i in range(len(neuron_type_list)):
        if neuron_type_list[i] == ON_OFF:
            neuron_keep = df_columns[i]
            new_df_column.append(neuron_keep)
    
    if 'Frame_Label' in new_df_column:
        new_df_column.remove('Frame_Label')
    elif 'status' in new_df_column:
        new_df_column.remove('status')
    
    new_df = df[new_df_column]
    new_df['Frame_Label'] = frame_label_list
    
    # shape(frames, keep_neurons+frame_label)
    return(new_df, new_df_column)


In [6]:
# generate ON/OFF summary file, save to csv
#----------------------------------------------------------------------------------
def generate_ONOFF_Summary(df, all_ONOFF_neuron_list, columns_name, mouse_name, project):
        
    all_neuron_list = df.columns.to_list()
    if 'Frame_Label' in all_neuron_list:
        all_neuron_list.remove('Frame_Label')
    
    label_list = []
    for i in all_neuron_list:
        label = []
        for a in range(len(all_ONOFF_neuron_list)):
            if i in all_ONOFF_neuron_list[a]:
                label.append(1)
            else:
                label.append(0)
        label_list.append(label)
        
    dic_ONOFF_summary = {}
    for i in range(len(all_neuron_list)):
        dic_ONOFF_summary[all_neuron_list[i]] = label_list[i]

    df_ONOFF_summary = pd.DataFrame.from_dict(dic_ONOFF_summary).T
    df_ONOFF_summary.columns = columns_name
    df_ONOFF_summary.to_csv(project+'_A_Summary_'+mouse_name+'_ONOFF.csv')

In [7]:
def degree(N, Na, Nb, Overlap):
    
    if Na == 0 and Nb == 0:
        observed_degree = 0
        predicted_degree = 0
    else:
        observed_degree = 2*Overlap/(Na+Nb)
        predicted_degree = 2*Na*Nb/(N*(Na+Nb))
    
    return(observed_degree, predicted_degree)

# Get all WT and NULL files

In [8]:
root_path = r'/Users/xiaoqiansun/Desktop/MedLu/EPM/EPM_Data'
root_path_incorrect = r'/Users/xiaoqiansun/Desktop/MedLu/EPM/EPM_Data/EPM-incorrect trace'

WT_list = ['NC128_WT', 'NC139_WT', 'NC158_WT', 'NC166_WT', 
           'NC228_WT', 'NC230_WT', 'NC238_WT', 'NC297_WT', 
           'NC298_WT', 'NC308_WT', 'NC314_WT', 'NC315_WT', 
           'NC324_WT', 'NC326_WT', 'NC463_WT', 'NC476_WT']
WT_incorrect_neuron_list = ['NC128_WT.txt', 'NC139_WT.txt', 'NC158_WT.txt', 'NC166_WT.txt',
                            'NC228_WT.txt', 'NC230_WT.txt', 'NC238_WT.txt', 'NC297_WT.txt', 
                            'NC298_WT.txt', 'NC308_WT.txt', 'NC314_WT.txt', 'NC315_WT.txt', 
                            'NC324_WT.txt', 'NC326_WT.txt',  'NC463_WT.txt', 'NC476_WT.txt']


#-----------------------------------------------------------------------------------------------

NULL_list = ['NC102_NULL', 'NC103_NULL', 'NC114_NULL', 'NC149_NULL', 
             'NC159_NULL', 'NC227_NULL', 'NC232_NULL', 'NC296_NULL', 
             'NC303_NULL', 'NC313_NULL', 'NC462_NULL', 'NC464_NULL', 
             'NC468_NULL','NC475_NULL']
NULL_incorrect_neuron_list = ['NC102_NULL.txt', 'NC103_NULL.txt', 'NC114_NULL.txt', 'NC149_NULL.txt', 
                              'NC159_NULL.txt', 'NC227_NULL.txt', 'NC232_NULL.txt', 'NC296_NULL.txt',
                              'NC303_NULL.txt', 'NC313_NULL.txt', 'NC462_NULL.txt', 'NC464_NULL.txt', 
                              'NC468_NULL.txt','NC475_NULL.txt']


# Generate Summary File

In [ ]:
print('WT')

for i in range(len(WT_list)):
    
    mouse_name = WT_list[i]
    
    mice = os.path.join(root_path, mouse_name)
    trace = os.path.join(root_path, mouse_name, "trace.csv")
    position = os.path.join(root_path, mouse_name, "cells position.csv")
    event = os.path.join(root_path, mouse_name, "events.xlsx")
    incorrect = os.path.join(root_path_incorrect, WT_incorrect_neuron_list[i])
    coordinate = os.path.join(root_path, mouse_name,'{}.TXT'.format(mouse_name.split('_')[0]))
    
    # defube fps based on mouse_name
    if mouse_name.split('_')[0] in ['NC128','NC102', 'NC013']:
        fps = 20
    else:
        fps = 15
        
    df = process_(trace, incorrect, position)
    df_frame_label = add_event_label(df, event, mice)
    
    # ON/OFF
    #####---------------------------------------------------------
    df_close_on , close_on_list  = status_ONOFF(df_frame_label, 'close', 'ON',  fps)
    df_close_off, close_off_list = status_ONOFF(df_frame_label, 'close', 'OFF', fps)
    
    df_open_on , open_on_list  = status_ONOFF(df_frame_label, 'open', 'ON',  fps)
    df_open_off, open_off_list = status_ONOFF(df_frame_label, 'open', 'OFF', fps)
    
    # Prefered ON/OFF
    #####---------------------------------------------------------
    Open_df_with_label  = check_frame_interval_with_label(df_frame_label, 'open')
    Close_df_with_label = check_frame_interval_with_label(df_frame_label, 'close')
    
    # get open prefered session
    open_x1, open_x2, open_p, open_np, OpenPrefered_Check, OpenNonPrefered_Check = prefered_side(df_frame_label, 
                                                                          Open_df_with_label,
                                                                          coordinate,
                                                                          fps)
    # get close prefered session
    close_x1, close_x2, close_p, close_np, ClosePrefered_Check, CloseNonPrefered_Check = prefered_side(df_frame_label,
                                                                            Close_df_with_label,
                                                                            coordinate,
                                                                            fps)
    #EPM openPrefered-NonPreferedON/closePrefered-NonPreferedON neuron list
    openPreferedON,      openPreferedON_list      = status_ONOFF_Prefered(OpenPrefered_Check,     'prefered',     fps, 'ON')
    openNonPreferedON,   openNonPreferedON_list   = status_ONOFF_Prefered(OpenNonPrefered_Check,  'non_prefered', fps, 'ON')
    closePreferedON,     closePreferedON_list     = status_ONOFF_Prefered(ClosePrefered_Check,    'prefered',     fps, 'ON')
    closeNonPreferedON,  closeNonPreferedON_list  = status_ONOFF_Prefered(CloseNonPrefered_Check, 'non_prefered', fps, 'ON')
    openPreferedOFF,     openPreferedOFF_list     = status_ONOFF_Prefered(OpenPrefered_Check,     'prefered',     fps, 'OFF')
    openNonPreferedOFF,  openNonPreferedOFF_list  = status_ONOFF_Prefered(OpenNonPrefered_Check,  'non_prefered', fps, 'OFF')
    closePreferedOFF,    closePreferedOFF_list    = status_ONOFF_Prefered(ClosePrefered_Check,    'prefered',     fps, 'OFF')
    closeNonPreferedOFF, closeNonPreferedOFF_list = status_ONOFF_Prefered(CloseNonPrefered_Check, 'non_prefered', fps, 'OFF')
    
    # generate on/off summary file for each mouse
    all_ONOFF_neuron_list = [open_on_list, open_off_list, close_on_list, close_off_list,
                             openPreferedON_list,    openPreferedOFF_list,
                             openNonPreferedON_list, openNonPreferedOFF_list,
                             closePreferedON_list,   closePreferedOFF_list,
                             closeNonPreferedON_list,closeNonPreferedOFF_list]
    columns_name = ['OpenON','OpenOFF', 'CloseON', 'CloseOFF',
                    'openPreferedON', 'openPreferedOFF', 'openNonPreferedON','openNonPreferedOFF',
                    'closePreferedON','closePreferedOF', 'closeNonPreferedON','closeNonPreferedOFF']
    generate_ONOFF_Summary(df, all_ONOFF_neuron_list, columns_name, mouse_name,'EPM')
    

In [51]:
print('NULL')

for i in range(len(NULL_list)):
    
    mouse_name = NULL_list[i]
    
    mice = os.path.join(root_path, mouse_name)
    trace = os.path.join(root_path, mouse_name, "trace.csv")
    position = os.path.join(root_path, mouse_name, "cells position.csv")
    event = os.path.join(root_path, mouse_name, "events.xlsx")
    incorrect = os.path.join(root_path_incorrect, NULL_incorrect_neuron_list[i])
    coordinate = os.path.join(root_path, mouse_name,'{}.TXT'.format(mouse_name.split('_')[0]))
    
    # defube fps based on mouse_name
    if mouse_name.split('_')[0] in ['NC128','NC102', 'NC013']:
        fps = 20
    else:
        fps = 15
        
    df = process_(trace, incorrect, position)
    df_frame_label = add_event_label(df, event, mice)
    
    # ON/OFF
    #####---------------------------------------------------------
    df_close_on , close_on_list  = status_ONOFF(df_frame_label, 'close', 'ON',  fps)
    df_close_off, close_off_list = status_ONOFF(df_frame_label, 'close', 'OFF', fps)
    
    df_open_on , open_on_list  = status_ONOFF(df_frame_label, 'open', 'ON',  fps)
    df_open_off, open_off_list = status_ONOFF(df_frame_label, 'open', 'OFF', fps)
    
    # Prefered ON/OFF
    #####---------------------------------------------------------
    Open_df_with_label  = check_frame_interval_with_label(df_frame_label, 'open')
    Close_df_with_label = check_frame_interval_with_label(df_frame_label, 'close')
    
    # get open prefered session
    open_x1, open_x2, open_p, open_np, OpenPrefered_Check, OpenNonPrefered_Check = prefered_side(df_frame_label, 
                                                                          Open_df_with_label,
                                                                          coordinate,
                                                                          fps)
    # get close prefered session
    close_x1, close_x2, close_p, close_np, ClosePrefered_Check, CloseNonPrefered_Check = prefered_side(df_frame_label,
                                                                            Close_df_with_label,
                                                                            coordinate,
                                                                            fps)
    #EPM openPrefered-NonPreferedON/closePrefered-NonPreferedON neuron list
    openPreferedON,      openPreferedON_list      = status_ONOFF_Prefered(OpenPrefered_Check,     'prefered',     fps, 'ON')
    openNonPreferedON,   openNonPreferedON_list   = status_ONOFF_Prefered(OpenNonPrefered_Check,  'non_prefered', fps, 'ON')
    closePreferedON,     closePreferedON_list     = status_ONOFF_Prefered(ClosePrefered_Check,    'prefered',     fps, 'ON')
    closeNonPreferedON,  closeNonPreferedON_list  = status_ONOFF_Prefered(CloseNonPrefered_Check, 'non_prefered', fps, 'ON')
    openPreferedOFF,     openPreferedOFF_list     = status_ONOFF_Prefered(OpenPrefered_Check,     'prefered',     fps, 'OFF')
    openNonPreferedOFF,  openNonPreferedOFF_list  = status_ONOFF_Prefered(OpenNonPrefered_Check,  'non_prefered', fps, 'OFF')
    closePreferedOFF,    closePreferedOFF_list    = status_ONOFF_Prefered(ClosePrefered_Check,    'prefered',     fps, 'OFF')
    closeNonPreferedOFF, closeNonPreferedOFF_list = status_ONOFF_Prefered(CloseNonPrefered_Check, 'non_prefered', fps, 'OFF')
    
    # generate on/off summary file for each mouse
    all_ONOFF_neuron_list = [open_on_list, open_off_list, close_on_list, close_off_list,
                             openPreferedON_list,    openPreferedOFF_list,
                             openNonPreferedON_list, openNonPreferedOFF_list,
                             closePreferedON_list,   closePreferedOFF_list,
                             closeNonPreferedON_list,closeNonPreferedOFF_list]
    columns_name = ['OpenON','OpenOFF', 'CloseON', 'CloseOFF',
                    'openPreferedON', 'openPreferedOFF', 'openNonPreferedON','openNonPreferedOFF',
                    'closePreferedON','closePreferedOF', 'closeNonPreferedON','closeNonPreferedOFF']
    generate_ONOFF_Summary(df, all_ONOFF_neuron_list, columns_name, mouse_name,'EPM')

NULL
57 neurons are not correct
136 neurons are not our target
141 neurons are removing
 
The df shape of trace df with label: (11800, 40)


/Users/xiaoqiansun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xiaoqiansun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/xiaoqiansun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

the shape of the dataframe in the status is: (3092, 40)
the shape of the dataframe in the status is: (8112, 40)
2064 frames are prefered
prefered 2064
non_prefered 1028
5176 frames are prefered
prefered 5176
non_prefered 2936


/Users/xiaoqiansun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


74 neurons are not correct
139 neurons are not our target
156 neurons are removing
 
The df shape of trace df with label: (11800, 25)
the shape of the dataframe in the status is: (1059, 25)
the shape of the dataframe in the status is: (9919, 25)
607 frames are prefered
prefered 607
non_prefered 452
7588 frames are prefered
prefered 7588
non_prefered 1337
122 neurons are not correct
55 neurons are not our target
149 neurons are removing
 
The df shape of trace df with label: (8925, 32)
the shape of the dataframe in the status is: (945, 32)
the shape of the dataframe in the status is: (7611, 32)
605 frames are prefered
prefered 605
non_prefered 340
5117 frames are prefered
prefered 5117
non_prefered 2218
83 neurons are not correct
156 neurons are not our target
171 neurons are removing
 
The df shape of trace df with label: (8925, 10)
the shape of the dataframe in the status is: (0, 10)
the shape of the dataframe in the status is: (8925, 10)
0 frames are prefered
prefered 0
non_prefered 

# Within Test Degree

In [9]:
root_path_on_off_summary = r'/Users/xiaoqiansun/Desktop/MedLu/EPM/EPM_Data/ON_OFF_Summary'

WT_degree_dic_list = []

for mouse_name in WT_list:
    
    project = 'EPM'
    EPM_file_name = project+'_A_Summary_'+mouse_name+'_ONOFF.csv'

    on_off_df = pd.read_csv(os.path.join(root_path_on_off_summary, EPM_file_name))
    columns = on_off_df.columns.to_list()
    columns[0] = 'Neurons'
    on_off_df.columns = columns
    on_off_df = on_off_df.set_index('Neurons')
    
    OpenON_CloseON_O, OpenON_CloseON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['OpenON'] == 1]),
                                                     len(on_off_df[on_off_df['CloseON'] == 1]),
                                                     len(on_off_df[(on_off_df['OpenON'] == 1) & (on_off_df['CloseON'] == 1)]))
    #------------
    OpenPON_OpenNPON_O, OpenPON_OpenNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['openNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openPreferedON'] == 1) & (on_off_df['openNonPreferedON'] == 1)]))
    #------------
    ClosePON_CloseNPON_O, ClosePON_CloseNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['closePreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closeNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['closePreferedON'] == 1) & (on_off_df['closeNonPreferedON'] == 1)]))
    #------------
    OpenON_ClosePON_O, OpenON_ClosePON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['OpenON'] == 1]),
                                                     len(on_off_df[on_off_df['closePreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['OpenON'] == 1) & (on_off_df['closePreferedON'] == 1)]))
    
    OpenON_CloseNPON_O, OpenON_CloseNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['OpenON'] == 1]),
                                                     len(on_off_df[on_off_df['closeNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['OpenON'] == 1) & (on_off_df['closeNonPreferedON'] == 1)]))
    #------------
    OpenPON_ClosePON_O, OpenPON_ClosePON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closePreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openPreferedON'] == 1) & (on_off_df['closePreferedON'] == 1)]))
    
    OpenPON_CloseNPON_O, OpenPON_CloseNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closeNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openPreferedON'] == 1) & (on_off_df['closeNonPreferedON'] == 1)]))
    #------------
    OpenNPON_ClosePON_O, OpenNPON_ClosePON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openNonPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closePreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openNonPreferedON'] == 1) & (on_off_df['closePreferedON'] == 1)]))
    
    OpenNPON_CloseNPON_O, OpenNPON_CloseNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openNonPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closeNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openNonPreferedON'] == 1) & (on_off_df['closeNonPreferedON'] == 1)]))
    
    degree_dic = {'mouse_name': mouse_name,
                  'OpenON_CloseON_O':    OpenON_CloseON_O,
                  'OpenON_CloseON_P':    OpenON_CloseON_P,
                  'OpenPON_OpenNPON_O':  OpenPON_OpenNPON_O,
                  'OpenPON_OpenNPON_P':  OpenPON_OpenNPON_P,
                  'ClosePON_CloseNPON_O':ClosePON_CloseNPON_O,
                  'ClosePON_CloseNPON_P':ClosePON_CloseNPON_P,
                  ' ': np.nan,
                  'OpenON_ClosePON_O':OpenON_ClosePON_O,
                  'OpenON_ClosePON_P':OpenON_ClosePON_P,
                  'OpenON_CloseNPON_O':OpenON_CloseNPON_O,
                  'OpenON_CloseNPON_P':OpenON_CloseNPON_P,
                  'OpenPON_ClosePON_O':OpenPON_ClosePON_O,
                  'OpenPON_ClosePON_P':OpenPON_ClosePON_P,
                  'OpenPON_CloseNPON_O':OpenPON_CloseNPON_O,
                  'OpenPON_CloseNPON_P':OpenPON_CloseNPON_P,
                  'OpenNPON_ClosePON_O':OpenNPON_ClosePON_O,
                  'OpenNPON_ClosePON_P':OpenNPON_ClosePON_P,
                  'OpenNPON_CloseNPON_O':OpenNPON_CloseNPON_O,
                  'OpenNPON_CloseNPON_P':OpenNPON_CloseNPON_P}
    degree_df = pd.DataFrame.from_dict(degree_dic, orient='index').T
    WT_degree_dic_list.append(degree_df)

WT_df_degree = pd.concat(WT_degree_dic_list)
WT_df_degree.to_csv('EPM_WT_Degree.csv')

In [10]:
root_path_on_off_summary = r'/Users/xiaoqiansun/Desktop/MedLu/EPM/EPM_Data/ON_OFF_Summary'

NULL_degree_dic_list = []

for mouse_name in NULL_list:
    
    project = 'EPM'
    EPM_file_name = project+'_A_Summary_'+mouse_name+'_ONOFF.csv'

    on_off_df = pd.read_csv(os.path.join(root_path_on_off_summary, EPM_file_name))
    columns = on_off_df.columns.to_list()
    columns[0] = 'Neurons'
    on_off_df.columns = columns
    on_off_df = on_off_df.set_index('Neurons')
    
    OpenON_CloseON_O, OpenON_CloseON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['OpenON'] == 1]),
                                                     len(on_off_df[on_off_df['CloseON'] == 1]),
                                                     len(on_off_df[(on_off_df['OpenON'] == 1) & (on_off_df['CloseON'] == 1)]))
    #------------
    OpenPON_OpenNPON_O, OpenPON_OpenNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['openNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openPreferedON'] == 1) & (on_off_df['openNonPreferedON'] == 1)]))
    #------------
    ClosePON_CloseNPON_O, ClosePON_CloseNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['closePreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closeNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['closePreferedON'] == 1) & (on_off_df['closeNonPreferedON'] == 1)]))
    #------------
    OpenON_ClosePON_O, OpenON_ClosePON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['OpenON'] == 1]),
                                                     len(on_off_df[on_off_df['closePreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['OpenON'] == 1) & (on_off_df['closePreferedON'] == 1)]))
    
    OpenON_CloseNPON_O, OpenON_CloseNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['OpenON'] == 1]),
                                                     len(on_off_df[on_off_df['closeNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['OpenON'] == 1) & (on_off_df['closeNonPreferedON'] == 1)]))
    #------------
    OpenPON_ClosePON_O, OpenPON_ClosePON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closePreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openPreferedON'] == 1) & (on_off_df['closePreferedON'] == 1)]))
    
    OpenPON_CloseNPON_O, OpenPON_CloseNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closeNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openPreferedON'] == 1) & (on_off_df['closeNonPreferedON'] == 1)]))
    #------------
    OpenNPON_ClosePON_O, OpenNPON_ClosePON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openNonPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closePreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openNonPreferedON'] == 1) & (on_off_df['closePreferedON'] == 1)]))
    
    OpenNPON_CloseNPON_O, OpenNPON_CloseNPON_P = degree(len(on_off_df.index),
                                                     len(on_off_df[on_off_df['openNonPreferedON'] == 1]),
                                                     len(on_off_df[on_off_df['closeNonPreferedON'] == 1]),
                                                     len(on_off_df[(on_off_df['openNonPreferedON'] == 1) & (on_off_df['closeNonPreferedON'] == 1)]))
    
    degree_dic = {'mouse_name': mouse_name,
                  'OpenON_CloseON_O':    OpenON_CloseON_O,
                  'OpenON_CloseON_P':    OpenON_CloseON_P,
                  'OpenPON_OpenNPON_O':  OpenPON_OpenNPON_O,
                  'OpenPON_OpenNPON_P':  OpenPON_OpenNPON_P,
                  'ClosePON_CloseNPON_O':ClosePON_CloseNPON_O,
                  'ClosePON_CloseNPON_P':ClosePON_CloseNPON_P,
                  ' ': np.nan,
                  'OpenON_ClosePON_O':OpenON_ClosePON_O,
                  'OpenON_ClosePON_P':OpenON_ClosePON_P,
                  'OpenON_CloseNPON_O':OpenON_CloseNPON_O,
                  'OpenON_CloseNPON_P':OpenON_CloseNPON_P,
                  'OpenPON_ClosePON_O':OpenPON_ClosePON_O,
                  'OpenPON_ClosePON_P':OpenPON_ClosePON_P,
                  'OpenPON_CloseNPON_O':OpenPON_CloseNPON_O,
                  'OpenPON_CloseNPON_P':OpenPON_CloseNPON_P,
                  'OpenNPON_ClosePON_O':OpenNPON_ClosePON_O,
                  'OpenNPON_ClosePON_P':OpenNPON_ClosePON_P,
                  'OpenNPON_CloseNPON_O':OpenNPON_CloseNPON_O,
                  'OpenNPON_CloseNPON_P':OpenNPON_CloseNPON_P}
    degree_df = pd.DataFrame.from_dict(degree_dic, orient='index').T
    NULL_degree_dic_list.append(degree_df)

NULL_df_degree = pd.concat(NULL_degree_dic_list)
NULL_df_degree.to_csv('EPM_NULL_Degree.csv')